# Google play store analysis.

# Training datasets are apps and user_reviews csv data files.

In [ ]:
# Importing necessary libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import pytz
%matplotlib inline

In [ ]:
# Importing the Apps and UserReviews datasets.

App = pd.read_csv("/content/Apps.csv")
UserReview = pd.read_csv("/content/User_reviews.csv")

**Data Cleaning Process**

In [ ]:
# Removing nan values from both datasets.

App.dropna(axis = "columns",how="any")
UserReview.dropna(axis = "columns",how="any")


# Converting Last Updated column into proper date format column.

App["Last Updated"] = pd.to_datetime(App["Last Updated"])

In [ ]:
# Merging the both datasets into one csv file data.

Data = pd.merge(App,UserReview,on="App",how="inner")        # Using merge function to combine the datasets. Hint : "inner" only gets the values matches on "App" key.


Data.to_csv("App_UserReview.csv",index="False")        # Converting csv file and skipping the index column through "False"


# Clean the "Installs" column: remove commas and '+' then convert to integer.

Data['Installs'] = Data['Installs'].replace('[+,]', '', regex=True).astype(int)


# Formatting Price column from string into float .

Data['Price'] = Data['Price'].str.replace("$","",regex=False).astype(float)


# Checking null values from the datasets.

# Rating, Current Ver, Android Ver, Translated_Review, Sentiment, Sentiment_Polarity, Sentiment_Subjectivity columns are null values occured by natures.

print(Data.isnull().sum())

App                           0
Category                      0
Rating                        0
Reviews                       0
Size                      20406
Installs                      0
Type                          0
Price                         0
Content Rating                0
Genres                        0
Last Updated                  0
Current Ver                   0
Android Ver                   0
Translated_Review         25627
Sentiment                 25622
Sentiment_Polarity        25622
Sentiment_Subjectivity    25622
dtype: int64


**Data Analysis Phase**

Task 1 :

Use a grouped bar chart to compare the average rating and total review count for the top 10 app categories by number of installs. Filter out any categories where the average rating is below 4.0 and size below 10 M and last update should be Jan month . this graph should work only between 3PM IST to 5 PM IST apart from that time we should not show this graph in dashboard itself.

In [ ]:
# Filter data the keep only rows where Rating >= 4.0, Size >= 10, and Last Updated month = January.

filter_d = Data[ (Data['Rating'] <= 4.0) & (Data['Size'] <= 10) & (Data['Last Updated'].dt.month == 1) ]


# Aggregate and get Top 10 categories by Installs

category_d = ( filter_d.groupby("Category", as_index=False)
        .agg({'Installs': 'sum', 'Rating': 'mean', 'Reviews': 'sum'})
        .sort_values(by='Installs', ascending=False)
        .head(10))


# Reshape using melt() function.

Top10_C = category_d.melt( id_vars='Category', value_vars=['Rating', 'Reviews'], var_name='Metric', value_name='Value')

print(Top10_C)

             Category   Metric      Value
0  LIBRARIES_AND_DEMO   Rating        3.5
1        PRODUCTIVITY   Rating        4.0
2              FAMILY   Rating        3.7
3             MEDICAL   Rating        3.8
4           EDUCATION   Rating        4.0
5  LIBRARIES_AND_DEMO  Reviews  5074480.0
6        PRODUCTIVITY  Reviews   359400.0
7              FAMILY  Reviews    95920.0
8             MEDICAL  Reviews    28840.0
9           EDUCATION  Reviews   101000.0


In [ ]:
# Set timezone
ist = pytz.timezone('Asia/Kolkata')
current_time = datetime.now(ist)

# Conditional plotting (only between 3 PM and 5 PM IST)
if 15 <= current_time.hour < 17 :

# Plot grouped bar chart.
   plt.figure(figsize=(12,9))
   sns.barplot(x="Category", y="Value", hue="Metric", data=Top10_C, color = "purple")
   plt.xticks(rotation=90)
   plt.title('Top 10 App Categories: Avg Rating & Total Reviews')
   plt.xlabel('App Category')
   plt.ylabel('Value')
   plt.tight_layout()
   plt.show()

else:
     print("📅 Chart not been showed - only allowed between 3PM and 5PM")

📅 Chart not been showed - only allowed between 3PM and 5PM


Task 2 :

Create an interactive Choropleth map using Plotly to visualize global installs by Category. Apply filters to show data for only the top 5 app categories and highlight category where the number of installs exceeds 1 million. The app category should not start with the characters “A,” “C,” “G,” or “S.” This graph should work only between 6 PM IST and 8 PM IST; apart from that time, we should not show it in the dashboard itself.

In [ ]:
# Filtering the installs exceeds 1000000 and category name not start with 'A','C','G','S'.

aci = Data [ ( Data['Installs'] > 1000000 ) & ~Data['Category'].str.startswith(('A','C','G','S')) ]


# Getting the top5 app category that have more than 1 millions installs and sort it

IC = ( aci.groupby('Category' , as_index = False)['Installs'].sum().sort_values(by='Installs',ascending=False) )
Top5_C = IC.head(5)


# Map categories to representative countries.

category_country_map = {
                       Top5_C.iloc[0]['Category']:'United States of America' ,
                       Top5_C.iloc[1]['Category']:'United States of Canada' ,
                       Top5_C.iloc[2]['Category']:'Russia' ,
                       Top5_C.iloc[3]['Category']:'China' ,
                       Top5_C.iloc[4]['Category']:'Australia'
                       }

# Creating country columns randomly in Top5_C .

Top5_C['Country'] = Top5_C['Category'].map(category_country_map)

print(Top5_C)

              Category      Installs                   Country
18        PRODUCTIVITY  338400000000  United States of America
17         PHOTOGRAPHY  297000000000   United States of Canada
19               TOOLS  197900000000                    Russia
15  NEWS_AND_MAGAZINES  186800000000                     China
20    TRAVEL_AND_LOCAL  108300000000                 Australia


/tmp/ipython-input-4115536567.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Top5_C['Country'] = Top5_C['Category'].map(category_country_map)


In [ ]:
# Visualize the choropleth map for Top5 app category their installs exceeds 1000000 and app word not start with 'A' , 'C' , 'G' , 'S'.


# Tme restriction 6 - 8 PM.

if 18 <= current_time.hour < 20:


# Visualizing the choropleth map.

  fig = px.choropleth(Top5_C , locations = 'Country' , locationmode = 'country names' , color = 'Installs' ,
                    hover_name='Category' , color_continuous_scale='plasma' , title = '🌍 Top 5 App Categories by Installs')


# Highlight categories exceeding 1M installs.

  for i, row in Top5_C.iterrows():
        fig.add_annotation(
            text=f"⭐ {row['Category']} ({row['Installs']:,} Installs)",
            showarrow=False,
            font=dict(color="black", size=15),
            xref="paper",
            yref="paper",
            x=0.5,
            y=1.05 - (i * 0.05) )
  fig.show()

else:
    print('📅 Chart not been showed - only allowed between 6PM and 8PM')

📅 Chart not been showed - only allowed between 6PM and 8PM


Task 3 :

Create a dual-axis chart comparing the average installs and revenue for free vs. paid apps within the top 3 app categories. Apply filters to exclude apps with fewer than 10,000 installs and revenue below $10,000 and android version should be more than 4.0 as well as size should be more than 15M and content rating should be Everyone and app name should not have more than 30 characters including space and special character. This graph should work only between 1 PM IST to 2 PM IST apart from that time we should not show this graph in dashboard itself.

In [ ]:
# To Create Revenue cloumn and multiplying Price into Installs columnns to generate each app revenue.

Data['Revenue'] = Data['Price'] * Data['Installs']
Data['Revenue'] = Data['Revenue'].astype(float)


# To filter the Android_Ver more than 4.0 version.

def is_greater_than_4_0(version_str):
    if pd.isna(version_str) or 'Varies' in version_str:
        return False
    try:
        # Converting the string (e.g., '4.0.3 and up' -> '4.0.3')
        version_part = version_str.split(' and up')[0].strip()
        version_part = version_part.split('-')[0].strip()


        # Split version into parts (Major.Minor.Patch)
        parts = [int(p) for p in version_part.split('.') if p.isdigit()]
        if not parts:
          return False

        major = parts[0]
        minor = parts[1] if len(parts) > 1 else 0
        patch = parts[2] if len(parts) > 2 else 0


        # Check if version > 4.0 (e.g., 4.1, 5.0, 4.0.1 are True, 4.0, 3.1 are False)
        return major > 4 or (major == 4 and (minor > 0 or patch > 0))
    except:
        return False

In [ ]:
# To named the particular filter data.

Data['Is_gt_4_0'] = Data['Android Ver'].apply(is_greater_than_4_0)
Data_AndVer = Data[Data['Is_gt_4_0']]


# To get an App character lesser than 30 character to filter out through the same version data.

Data_AndroidVer = Data_AndVer[Data_AndVer['App'].str.len() <= 30]



# To filter Installs and Revenue more than 10000, Size greater than 15 and Content Rating is Everyone from the version data.

DA = Data_AndroidVer[ ( Data_AndroidVer['Installs'] >= 10000 ) & (Data_AndroidVer['Revenue'] >= 10000) & (Data_AndroidVer['Size'] > 15)  &
                     (Data_AndroidVer['Content Rating'] == 'Everyone') ]



# Get the top 3 category by installs.

q = (DA.groupby('Category')['Installs'].sum().nlargest(3).index)

r = DA[DA['Category'].isin(q)]


# Calculate the average installs and average revenue for paid and free type.

Top3_Category = r.groupby("Type").agg( avg_inst = ("Installs","mean"), avg_rev = ("Revenue","mean") ).reset_index()

In [ ]:
# Visualize the Dual-Axis Chart.

ist = pytz.timezone('Asia/Kolkata')
current_time = datetime.now(ist)


# Time restriction (1 PM – 2 PM IST)

if 13 <= current_time.hour < 14 :


   fig,ax1 = plt.subplots(figsize=(10, 6))


# Bar plot – Avg Installs

   ax1.bar(Top3_Category["Type"] , Top3_Category["avg_inst"], width=0.3, color = 'purple')
   ax1.set_ylabel("Average Installs")
   ax1.set_xlabel("App Type (Free vs Paid)")
   ax1.set_title("Dual-Axis Chart: Avg Installs & Revenue for Free vs Paid Apps")


# Second Axis – Avg Revenue

   ax2 = ax1.twinx()
   ax2.plot(Top3_Category["Type"], Top3_Category["avg_rev"], marker='p', linewidth=2)
   ax2.set_ylabel("Average Revenue ($)")

   plt.tight_layout()
   plt.show()

else:
   print('📅 Chart not been showed - only allowed between 1PM and 2PM')

📅 Chart not been showed - only allowed between 1PM and 2PM


Task 4 :

Plot a time series line chart to show the trend of total installs over time, segmented by app category. Highlight periods of significant growth by shading the areas under the curve where the increase in installs exceeds 20% month-over-month and app name should not starts with x, y ,z and app category should start with letter " E " or " C " or " B " and We have to translate the Beauty category in Hindi and Business category in Tamil and Dating category in German while showing it on Graph. reviews should be more than 500 the app name should not contain letter "S" as well as this graph should work only between 6 PM IST to 9 PM IST apart from that time we should not show this graph in dashboard itself

In [ ]:
# Extracting the month from Last Updated column to get the monthly results.

Data["Month"] = Data['Last Updated'].dt.to_period("M").dt.to_timestamp()


# Filter the app name letter contains 'S' , not contains 'x','y','z' letters and category letters contains 'E','C','B' , reviews are greater than 500.

d = Data[(~Data['App'].str.startswith(("x","y","z"),na=False)) & ( Data['Category'].str.startswith(("E","C","B"),na=False) ) &
        ( (Data['Reviews'] > 500 ) & ~Data['App'].str.contains("S", na = True) ) ]



# Translating the category from beauty, business and dating into सौंदर्य , வணிகம் , Dating.

category_translation = {  'BEAUTY': 'सौंदर्य',      # Hindi
                             'BUSINESS': 'வணிகம்',     # Tamil
                            'DATING': 'Dating'                # Germany
                           }

d["Category"] = d['Category'].replace(category_translation)



# Aggregate the total installs by category and month.


monthly_installs = d.groupby(['Month','Category'])['Installs'].sum().reset_index()

monthly_installs['M_G'] = monthly_installs.groupby('Category')['Installs'].pct_change()

/tmp/ipython-input-3447835985.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["Category"] = d['Category'].replace(category_translation)


In [ ]:
# Time Restriction between 6 PM to 9 PM.

if 18 <= current_time.hour < 21:


    # Time Series Graph.

    plt.figure(figsize=(12, 6))

    categories = monthly_installs['Category'].unique()
    for cat in categories:
        cat_data = monthly_installs[monthly_installs['Category'] == cat]

        plt.plot(cat_data['Month'], cat_data['Installs'], linewidth=2, label=cat)


        # Shade where MoM Growth > 20%

        growth_mask = cat_data['M_G'] > 0.20
        plt.fill_between(cat_data['Month'], cat_data['Installs'], where= growth_mask, alpha=0.3)


    # labelling the graph axis and legend.

    plt.title("Time Series Trend of Total Installs by Category\n(Shaded Areas = >20% Monthly Growth)")
    plt.xlabel("Month")
    plt.ylabel("Total Installs")
    plt.grid(True)
    plt.legend(title="Categories Names")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

else:
    print("📅 Chart not been showed - only allowed between 6PM and 9PM")

📅 Chart not been showed - only allowed between 6PM and 9PM


Task 5 :

Plot a bubble chart to analyze the relationship between app size (in MB) and average rating, with the bubble size representing the number of installs. Include a filter to show only apps with a rating higher than 3.5 and that belong to the Game, Beauty ,business , commics , commication , Dating , Entertainment , social and event categories. Reviews should be greater than 500 and the app name should not contain letter "S" and sentiment subjectivity should be more than 0.5 and highlight the Game Category chart in Pink color. We have to translate the Beauty category in Hindi and Business category in Tamil and Dating category in German while showing it on Graphs. Installs should be more than 50k as well as this graph should work only between 5 PM IST to 7 PM IST apart from that time we should not show this graph in dashboard itself.

In [ ]:
# Translating the category from beauty, business and dating into hindi, tamil and german languages.

category_translation = {  'BEAUTY': 'सौंदर्य',      # Hindi
                             'BUSINESS': 'வணிகம்',     # Tamil
                            'DATING': 'Dating'                # Germany
                        }

# Filterout the conditions that matches above criterias.

valid_categories   =  ['GAME','BUSINESS','BEAUTY','COMICS','COMMUNICATION','DATING','ENTERTAINMENT','SOCIAL','EVENTS']


Filtered_dt = Data [ ( (Data['Rating'] > 3.5) & (Data['Category'].isin(valid_categories))  & (Data['Reviews'] > 500) & (~Data['App'].str.contains("S"))
             & (Data['Sentiment_Subjectivity'] > 0.5) & (Data['Installs'] > 50000) ) ]


Filtered_dt['Category'] = Filtered_dt['Category'].replace(category_translation)

/tmp/ipython-input-3664277135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Filtered_dt['Category'] = Filtered_dt['Category'].replace(category_translation)


In [ ]:
# Time Restriction between 5 PM to 7 PM.

if 17 <= current_time.hour < 19:


# Visualize the bubble chart relationship beteween App size and Average rating by Installs.

    plt.figure(figsize=(10,8))

    Game_cate = Filtered_dt[Filtered_dt['Category'] == 'GAME' ]
    Other_cate = Filtered_dt[Filtered_dt['Category'] != 'GAME' ]


    plt.scatter(Other_cate['Size'], Other_cate['Rating'] , s = Other_cate['Installs'] / 4247 , color = 'green', edgecolors='black', label = 'Other Categories' )

    plt.scatter(Game_cate['Size'], Game_cate['Rating'] , s = Game_cate['Installs'] / 4247 , color = 'pink', edgecolors='black', label = 'Game Categories' )


    plt.xlabel('App Size(MB)')
    plt.ylabel('Average Rating')
    plt.title("Bubble Chart: App Size vs Rating by Installs in bubble size")
    plt.grid(True)
    plt.show()

else:
    print("📅 Chart not been showed - only allowed between 5PM and 7PM")

📅 Chart not been showed - only allowed between 5PM and 7PM
